In [ ]:
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np
from keras import objectives
from keras import backend as K
from keras import losses
import tensorflow as tf
import interactions_results
import train_interactions

In [ ]:
OBJ_IDS = ['1', '2']

COLUMNS_MAP = [('x', 'ant%s_x'),
       ('y', 'ant%s_y'),
       ('major', 'ant%s_major'),
       ('minor', 'ant%s_minor'),
       ('angle_deg', 'ant%s_angle_deg'),
       ('dx', 'ant%s_dx'),
       ('dy', 'ant%s_dy'),
       ]

COL = dict(COLUMNS_MAP)
NAMES = reduce(list.__add__, [[value % i for key, value in COLUMNS_MAP] for i in OBJ_IDS])
COL2ID = {key: i for i, (key, value) in enumerate(COLUMNS_MAP)}

In [ ]:
COL2ID

In [ ]:
def angle_absolute_error(y_true, y_pred, backend, scaler=None):
    if scaler is not None:
        # y_pred_ = scaler.inverse_transform(y_pred[:, 4:5])  # this doesn't work with Tensors
        y_pred_ = y_pred[:, 4:5] * scaler[1] + scaler[0]
    else:
        y_pred_ = y_pred[:, 4:5]
    val = backend.abs(y_pred_ - y_true[:, 4:5]) % 180
    return backend.minimum(val, 180 - val)


def xy_absolute_error(y_true, y_pred, backend):
    return backend.abs(y_pred[:, :2] - y_true[:, :2])


def absolute_errors(y_true, y_pred, backend, angle_scaler):
    theta = angle_absolute_error(y_true, y_pred, backend, angle_scaler)
    pos = xy_absolute_error(y_true, y_pred, backend)
    return pos, theta


def interaction_loss(y_true, y_pred, angle_scaler=None, alpha=0.5):
    assert 0 <= alpha <= 1
    sum_errors_xy, sum_errors_angle, indices = match_pred_to_gt(y_true, y_pred, K, angle_scaler)

    return K.mean(tf.gather_nd(sum_errors_xy, indices) * (1 - alpha) +
                  tf.gather_nd(sum_errors_angle, indices) * alpha)


y_a = np.array([[10., 10, 25, 5, 20, 100, 100, 25, 5, 30],
                [100., 100, 25, 5, 30, 20, 20, 25, 5, 20],
                [10., 10, 25, 5, 20, 200, 200, 25, 5, 30]])
y_b = np.array([[20., 20, 25, 5, 30, 150, 170, 25, 5, 0],
                [30., 30, 25, 5, 30, 170, 150, 25, 5, 5],
                [30., 60, 25, 5, 30, 170, 120, 25, 5, 5]])
xy, angle, indices = train_interactions.match_pred_to_gt(y_a, y_b, np)

print (xy[indices[:, 0], indices[:, 1]]).mean()
print (angle[indices[:, 0], indices[:, 1]]).mean()

In [ ]:
# with h5py.File(DATA_DIR + '/imgs_inter_test.h5', 'r') as hf:
#     X_test = hf['data'][:]
#
y_a_ = interactions_results.tostruct(y_a)
y_b_ = interactions_results.tostruct(y_b)
i = 1
interactions_results.plot_interaction(y_a_[[i]], y_b_[[i]])
plt.ylim(0, 200)
plt.xlim(0, 200)

In [ ]:
y_true = K.variable(y_a)
y_pred = K.variable(y_b)
backend = K
angle_scaler = None

In [ ]:
K.eval(y_pred[:, 1:2])

In [ ]:
K.eval(y_pred[:, [COL2ID['x'], COL2ID['y']]] - y_true[:, [COL2ID['x'], COL2ID['y']]])

In [ ]:
y_true = y_a
y_pred = y_b
backend = np
angle_scaler = None

In [ ]:
mean_errors_xy, mean_errors_angle, indices = train_interactions.match_pred_to_gt(y_true, y_pred, backend)
for x in [mean_errors_xy, mean_errors_angle, indices]:
    print x

In [ ]:
mean_errors_xy, mean_errors_angle, indices = train_interactions.match_pred_to_gt(y_true, y_pred, backend)
for x in [mean_errors_xy, mean_errors_angle, indices]:
    print K.eval(x)

In [ ]:
mean_errors_xy, mean_errors_angle, indices = train_interactions.match_pred_to_gt(y_true, y_pred, K)
for x in [mean_errors_xy, mean_errors_angle, indices]:
    print K.eval(x)

In [ ]:
# def match_pred_to_gt(y_true, y_pred, backend, angle_scaler=None):
"""
Return mean absolute errors for individual samples for xy and theta
in two possible combinations of prediction and ground truth.
"""
xy11, theta11 = absolute_errors(y_true[:, :5], y_pred[:, :5], backend, angle_scaler)
xy22, theta22 = absolute_errors(y_true[:, 5:], y_pred[:, 5:], backend, angle_scaler)
xy12, theta12 = absolute_errors(y_true[:, :5], y_pred[:, 5:], backend, angle_scaler)
xy21, theta21 = absolute_errors(y_true[:, 5:], y_pred[:, :5], backend, angle_scaler)
if backend == np:
    norm = np.linalg.norm
    int64 = np.int64
    shape = lambda x, n: x.shape[n]
else:
    norm = tf.linalg.norm
    int64 = tf.int64
    shape = lambda x, n: backend.cast(backend.shape(x)[n], int64)
mean_errors_xy = backend.stack((backend.mean(backend.stack((norm(xy11, axis=1), norm(xy22, axis=1))), axis=0),
                               backend.mean(backend.stack((norm(xy12, axis=1), norm(xy21, axis=1))), axis=0)))  # shape=(2, n)

mean_errors_angle = backend.stack((backend.mean(backend.concatenate((theta11, theta22)), axis=1),
                                  backend.mean(backend.concatenate((theta12, theta21)), axis=1)))  # shape=(2, n)

print K.eval(theta11)
print K.eval(backend.concatenate((theta11, theta22)))
print K.eval(backend.sum(backend.concatenate((theta11, theta22)), axis=1))

swap_idx = backend.argmin(mean_errors_xy, axis=0)  # shape = (n,)
indices = backend.transpose(
    backend.stack((swap_idx, backend.arange(0, shape(mean_errors_xy, 1)))))  # shape=(n, 2)
# return mean_errors_xy, mean_errors_angle, indices

for x in [mean_errors_xy, mean_errors_angle, indices]:
    print K.eval(x)

In [ ]:
angle_scaler = None
y_true = K.variable(y_a)
y_pred = K.variable(y_b)
xy11, theta11 = absolute_errors(y_true[:, :5], y_pred[:, :5], angle_scaler)
xy22, theta22 = absolute_errors(y_true[:, 5:], y_pred[:, 5:], angle_scaler)

xy12, theta12 = absolute_errors(y_true[:, :5], y_pred[:, 5:], angle_scaler)
xy21, theta21 = absolute_errors(y_true[:, 5:], y_pred[:, :5], angle_scaler)

norm = tf.linalg.norm
# print y_a
# print y_b
# print K.eval(xy11)
# print K.eval(xy22)
# print K.eval(xy12)
# print K.eval(xy21)
sum_errors_xy = K.stack((K.sum(K.stack((norm(xy11, axis=1), norm(xy22, axis=1))), axis=0),
                         K.sum(K.stack((norm(xy12, axis=1), norm(xy21, axis=1))), axis=0)))  # shape=(2, n)
sum_errors_angle = K.stack((K.sum(K.concatenate((theta11, theta22)), axis=1),
                            K.sum(K.concatenate((theta12, theta21)), axis=1)))  # shape=(2, n)

swap_idx = K.argmin(sum_errors_xy, axis=0)  # shape = (n,)
indices = K.transpose(K.stack((swap_idx, K.arange(0, K.cast(K.shape(sum_errors_xy)[1], tf.int64)))))  # shape=(n, 2)  

print K.eval(tf.gather_nd(sum_errors_xy, idx))
print K.eval(tf.gather_nd(sum_errors_angle, idx))
print K.eval(tf.gather_nd(sum_errors_xy, idx) + tf.gather_nd(sum_errors_angle, idx))

In [ ]:
idx = K.transpose(K.stack((swap_idx, K.arange(0, K.cast(K.shape(sum_errors_xy)[1], tf.int64)))))

In [ ]:
K.eval(tf.gather_nd(sum_errors_xy, idx))

In [ ]:
K.eval(sum_errors_xy)

In [ ]:
K.eval(sum_errors_angle)

In [ ]:
K.eval(tf.gather_nd(sum_errors_angle, idx))

In [ ]:
np.ca